<a href="https://colab.research.google.com/github/durfred/my-first-binder/blob/main/chess_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download Gemini GGUF (example: Gemini-1.5-mini Q4_0 GGUF)
!wget -O gemini.gguf https://huggingface.co/bartowski/gemma-2-2b-it-abliterated-GGUF/resolve/main/gemma-2-2b-it-abliterated-IQ4_XS.gguf

In [ ]:
!pip install llama-cpp-python



In [40]:
import re

# Clean Gemini output
def clean_move_string(raw):
    move = raw.strip().lower()
    move = re.sub(r'[^a-z0-9=]+', '', move)  # Remove non-uci/san characters (like **, .)
    return move

def ai_move_with_gemini(board):
    prompt = f"""
You are a chess engine. Only respond with a legal move in **UCI format** (e.g., e2e4, g1f3). Do not explain. No SAN like 'e4'. Just the move.

FEN: {board.fen()}

Move:"""

    output = llm(prompt, max_tokens=16, temperature=0.2, stop=["\n"])
    move_str = output['choices'][0]['text']
    print(f"Gemini raw output: {move_str}")

    move_str_clean = clean_move_string(move_str)

    # Try UCI first
    try:
        move = chess.Move.from_uci(move_str)
        if move in board.legal_moves:
            return move
    except:
        pass

    # Try SAN fallback with title-casing (e.g., d3 -> D3)
    try:
        move_str_san = move_str.strip().title()
        move = board.parse_san(move_str_san)
        if move in board.legal_moves:
            print(f"✔ Interpreted SAN '{move_str_san}' as UCI '{move.uci()}'")
            return move
    except Exception as e:
        print(f"❌ Failed to interpret move: {e}")

    return None


In [36]:
# test_board = chess.Board()
# print("FEN:", test_board.fen())
# print("Gemini says:", ai_move_with_gemini(test_board))


FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1


Llama.generate: 6 prefix-match hit, remaining 88 prompt tokens to eval
llama_perf_context_print:        load time =   14716.68 ms
llama_perf_context_print: prompt eval time =   15808.76 ms /    88 tokens (  179.65 ms per token,     5.57 tokens per second)
llama_perf_context_print:        eval time =     310.75 ms /     1 runs   (  310.75 ms per token,     3.22 tokens per second)
llama_perf_context_print:       total time =   16126.06 ms /    89 tokens


Gemini raw output: 
❌ Failed to interpret move
Gemini says: None
